# DDSP-SVC 추론 전용 코랩



In [1]:
# 레포지토리 클론, 필요한 라이브러리 설치
#***설치 후 restart 필요!!***

!git clone https://github.com/yxlllc/DDSP-SVC.git
%cd DDSP-SVC
!pip install soundfile
!pip install gdown
!pip install tqdm
!pip install -r requirements.txt
!pip install numpy==1.23.5
print("\n\n▶레포지토리 클론과 라이브러리 설치 완료◀")

Cloning into 'DDSP-SVC'...
remote: Enumerating objects: 755, done.
remote: Counting objects: 100% (483/483), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 755 (delta 356), reused 358 (delta 291), pack-reused 272
Receiving objects: 100% (755/755), 20.76 MiB | 28.50 MiB/s, done.
Resolving deltas: 100% (455/455), done.
/content/DDSP-SVC
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 71.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... d



▶레포지토리 클론과 라이브러리 설치 완료◀


In [1]:
# 필요한 파일 다운로드

%cd /content/DDSP-SVC

# contentvec
contentvec_link = 'https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt'
contentvec_dir = './pretrain/ContentVec'
!wget {contentvec_link} -P {contentvec_dir}

nsf_model_link = 'https://huggingface.co/spaces/ulysses115/diffsvc_test/resolve/main/checkpoints/nsf_hifigan/model'
nsf_config_link = 'https://huggingface.co/spaces/ulysses115/diffsvc_test/raw/main/checkpoints/nsf_hifigan/config.json'
nsf_dir = './pretrain/nsf_hifigan'
!wget {nsf_model_link} -P {nsf_dir}
!wget {nsf_config_link} -P {nsf_dir}

hubert_link = 'https://huggingface.co/spaces/ulysses115/diffsvc_test/resolve/main/checkpoints/hubert/hubert_soft.pt'
hubert_path = './pretrain/hubert/hubert-soft-0d54a1f4.pt'
!wget {hubert_link} -O {hubert_path}

print("\n\n▶필요한 파일 다운로드 완료◀")

/content/DDSP-SVC
--2023-05-05 05:51:35--  https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt
Resolving huggingface.co (huggingface.co)... 13.226.22.54, 13.226.22.117, 13.226.22.112, ...
Connecting to huggingface.co (huggingface.co)|13.226.22.54|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/0a/5f/0a5f89ff9058782f4b7a207d38e0914d63f6f55e8fc1fad0c3bf68b546039f2d/60d936ec5a566776fc392e69ad8b630d14eb588111233fe313436e200a7b187b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27checkpoint_best_legacy_500.pt%3B+filename%3D%22checkpoint_best_legacy_500.pt%22%3B&Expires=1683525095&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzBhLzVmLzBhNWY4OWZmOTA1ODc4MmY0YjdhMjA3ZDM4ZTA5MTRkNjNmNmY1NWU4ZmMxZmFkMGMzYmY2OGI1NDYwMzlmMmQvNjBkOTM2ZWM1YTU2Njc3NmZjMzkyZTY5YWQ4YjYzMGQxNGViNTg4MTExMjMzZmUzMTM0MzZlMjAwYTdiMTg3Yj9yZXNwb25zZS1jb250ZW50LWRpc3

In [2]:
from google.colab import drive
drive.mount('/content/drive')
print("\n\n▶구글 드라이브 마운트 완료◀")

Mounted at /content/drive


▶구글 드라이브 마운트 완료◀


In [4]:
#@markdown # 기본 설정

#@markdown ### 모델 이름
model_name = "submission" #@param {type:"string"}

#@markdown ### 모델 파일 경로
#@markdown - 이전에 학습한 ddsp-svc 모델 파일 경로를 기입해주세요
checkpoint_path = "/content/drive/MyDrive/chatgpt\u110B\u1161\u11AB\u110A\u1165\u11BA\u110C\u1169/\u1100\u1175\u1100\u1168\u1112\u1161\u11A8\u1109\u1173\u11B8\u1100\u116A\u1103\u1175\u11B8\u1105\u1165\u1102\u1175\u11BC_\u110B\u1175\u110C\u1162\u110B\u116E\u11A8\u1100\u116D\u1109\u116E\u1102\u1175\u11B7/Final_Project/diff-svc/train/gyeongju/model_best.pt" #@param {type:"string"}

#@ markdown ## 구글 드라이브 작업 경로를 입력하세요.
#@markdown - 아래와 같이 입력하면
#@markdown - /content/drive/MyDrive/ddsp-svc
#@markdown - 구글드라이브의 최상위 디렉터리에 ddsp-svc 폴더를 사용하는 것입니다.
work_path = "/content/drive/MyDrive/chatgpt\u110B\u1161\u11AB\u110A\u1165\u11BA\u110C\u1169/\u1100\u1175\u1100\u1168\u1112\u1161\u11A8\u1109\u1173\u11B8\u1100\u116A\u1103\u1175\u11B8\u1105\u1165\u1102\u1175\u11BC_\u110B\u1175\u110C\u1162\u110B\u116E\u11A8\u1100\u116D\u1109\u116E\u1102\u1175\u11B7/Final_Project/diff-svc/inference" #@param {type:"string"}

import os
import shutil
assert os.path.isfile(checkpoint_path), '모델 파일 경로가 올바르지 않습니다.'

!rm -rf /content/DDSP-SVC/exp/*

shutil.copy(checkpoint_path, '/content/DDSP-SVC/exp/model_best.pt')
shutil.copy('/content/DDSP-SVC/configs/combsub.yaml', '/content/DDSP-SVC/exp/config.yaml')

import yaml

with open('configs/combsub.yaml') as f:
  config = yaml.load(f, Loader=yaml.FullLoader)

config['data']['encoder'] = 'contentvec'
config['data']['encoder_ckpt'] = './pretrain/ContentVec/checkpoint_best_legacy_500.pt'
config['train']['cache_device'] = 'cuda'
config['env']['expdir'] = f'./exp'

with open('configs/combsub.yaml', 'w') as f:
  yaml.dump(config, f)
with open('exp/config.yaml', 'w') as f:
  yaml.dump(config, f)

print('▶설정 저장 완료◀')

▶설정 저장 완료◀


In [5]:
from glob import glob
import subprocess
import torchaudio
import torch
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB_PLUS
from torchaudio.transforms import Fade
import soundfile as sf
from tqdm import tqdm
from glob import glob
import shutil
import librosa
from IPython import display as ipd

#@markdown ## 음원분리 모듈 로드
#@markdown - 음원분리용 모듈을 로드합니다.

get_filename = lambda f: os.path.basename(f)

class demucs():
  def __init__(self):
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    bundle = HDEMUCS_HIGH_MUSDB_PLUS
    self.model = bundle.get_model()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.model.to(self.device)
    self.sample_rate = bundle.sample_rate
  
  def separate_sources(
        self,
        model,
        mix,
        segment=10.,
        overlap=0.1,
        device=None,
        sample_rate=44100,
):
    """
    Apply model to a given mixture. Use fade, and add segments together in order to add model segment by segment.

    Args:
        segment (int): segment length in seconds
        device (torch.device, str, or None): if provided, device on which to
            execute the computation, otherwise `mix.device` is assumed.
            When `device` is different from `mix.device`, only local computations will
            be on `device`, while the entire tracks will be stored on `mix.device`.
    """
    if device is None:
        device = mix.device
    else:
        device = torch.device(device)

    batch, channels, length = mix.shape

    chunk_len = int(sample_rate * segment * (1 + overlap))
    start = 0
    end = chunk_len
    overlap_frames = overlap * sample_rate
    fade = Fade(fade_in_len=0, fade_out_len=int(overlap_frames), fade_shape='linear')

    final = torch.zeros(batch, len(model.sources), channels, length, device=device)

    while start < length - overlap_frames:
        chunk = mix[:, :, start:end]
        with torch.no_grad():
            out = model.forward(chunk)
        out = fade(out)
        final[:, :, :, start:end] += out
        if start == 0:
            fade.fade_in_len = int(overlap_frames)
            start += int(chunk_len - overlap_frames)
        else:
            start += chunk_len
        end += chunk_len
        if end >= length:
            fade.fade_out_len = 0
    return final
  
  def extract_vocal(self, input_file_path, output_folder_path):
    waveform, self.sample_rate = torchaudio.load(input_file_path)
    waveform = waveform.to(self.device)
    mixture = waveform

    # parameters
    segment: int = 10
    overlap = 0.1

    ref = waveform.mean(0)
    waveform = (waveform - ref.mean()) / ref.std()  # normalization

    sources = self.separate_sources(
        self.model,
        waveform[None],
        device=self.device,
        segment=segment,
        overlap=overlap,
    )[0]

    sources = sources * ref.std() + ref.mean()

    sources_list = self.model.sources
    sources = list(sources)

    audios = dict(zip(sources_list, sources))
    sf.write(os.path.join(output_folder_path, 'seperate.wav'), audios['vocals'][0].cpu().numpy(), self.sample_rate)
    sf.write(os.path.join(output_folder_path, 'other.wav'), audios['other'][0].cpu().numpy(), self.sample_rate)
    sf.write(os.path.join(output_folder_path, 'drums.wav'), audios['drums'][0].cpu().numpy(), self.sample_rate)
    sf.write(os.path.join(output_folder_path, 'bass.wav'), audios['bass'][0].cpu().numpy(), self.sample_rate)

seperator = demucs()

100%|██████████| 319M/319M [00:03<00:00, 96.4MB/s]


In [6]:
# 사용자로부터 추론에 사용할 레퍼런스 음원 업로드 창 띄우기
#@markdown ## 원곡의 파일을 업로드합니다.
#@markdown ### 원곡 파일은 다음의 조건을 충족해야 합니다.
#@markdown - 노래를 부르는 사람이 한 명이어야 합니다. (필수)
#@markdown - 파일은 mp3나 wav형식이어야 합니다. (필수)
#@markdown - 노래에 코러스같은 화음이 포함되지 않으면 좋습니다. (선택)
#@markdown - 피아노나 기타 드럼 등 알려진 악기는 포함되어도 좋습니다.
#@markdown - 그러나 edm 효과음 등의 커스텀 악기 알려지지 않은 악기가 포함되지 않으면 좋습니다.(선택)
#@markdown
#@markdown ### 모든 항목을 준수하지 않아도 결과물은 나옵니다. 하지만 좋은 결과물은은 아닐겁니다.

from google.colab import files
from datetime import datetime

ipd.clear_output()

uploaded = files.upload()
assert len(uploaded) == 1, "파일을 하나만 업로드해주세요."
upload_file_name = list(uploaded.keys())[0]
assert upload_file_name.endswith(".mp3") or upload_file_name.endswith(".wav"), "mp3나 wav파일만 업로드해주세요."
reference_file_path = os.path.join(os.getcwd(), upload_file_name)

currunt_time = datetime.now().strftime('%Y-%m-%d_%H-%M')
infer_dir = os.path.join('infer', model_name, upload_file_name.split('.')[0] + '_' + currunt_time)
os.makedirs(infer_dir, exist_ok=True)

drive_infer_dir = os.path.join(work_path, model_name, upload_file_name.split('.')[0] + '_' + currunt_time)
os.makedirs(drive_infer_dir, exist_ok=True)

seperate_output_path = infer_dir
seperator.extract_vocal(reference_file_path, seperate_output_path)

shutil.copy(reference_file_path, drive_infer_dir)
shutil.copytree(seperate_output_path, drive_infer_dir, dirs_exist_ok=True)

print('업로드한 파일로부터 보컬을 분리했습니다')
print('분리된 보컬을 띄우긴 했으나 보이지 않으면 구글드라이브 폴더를 확인해보세요.')

Saving madeleinelove_cut2.mp3 to madeleinelove_cut2.mp3
업로드한 파일로부터 보컬을 분리했습니다
분리된 보컬을 띄우긴 했으나 보이지 않으면 구글드라이브 폴더를 확인해보세요.


In [7]:
# 추론 실행하고, 코랩에 추론된 결과물을 보여주고, 추론 결과를 구글 드라이브에 저장하기
#@markdown ## 추론
#@markdown 추론을 수행합니다
#@markdown 키(음성의 음높낮이)를 바꾸려면 아래 변수를 바꾸어주세요
import pydub
key = -3 #@param {type:"integer"}
seperate_vocal_path = os.path.join(seperate_output_path, 'seperate.wav')
infer_output_path = os.path.join(infer_dir, f'infered_output_key{key}.wav')
command = f"python main.py -i {seperate_vocal_path} -m /content/DDSP-SVC/exp/model_best.pt -o {infer_output_path} -k {key} -id 1 -eak 0"
subprocess.run(command, shell=True)

shutil.copy(infer_output_path, drive_infer_dir)
print('분리한 보컬로 추론이 완료되었습니다.')
print('추론된 보컬을 띄우긴 했으나 보이지 않으면 구글드라이브 폴더를 확인해보세요.')

분리한 보컬로 추론이 완료되었습니다.
추론된 보컬을 띄우긴 했으나 보이지 않으면 구글드라이브 폴더를 확인해보세요.


In [12]:
#@title 보컬 제외한 악기들의 키 조절 (pitch shift)
import soundfile as sf

inst_list = ['bass', 'drums', 'other']

#shifting_key = key
shifting_key = -3 #@param {type:"raw"}
drive_infer_dir = drive_infer_dir #@param {type:"raw"}

if shifting_key != 0:
  for inst in inst_list:
    y, sample_rate = librosa.load(os.path.join(drive_infer_dir, inst + '.wav'), sr=44100)
    y_shifted = librosa.effects.pitch_shift(y, sample_rate, n_steps=shifting_key)
    shifted = inst + f'_shifted_{shifting_key}.wav'
    sf.write(os.path.join(drive_infer_dir, shifted), y_shifted, 44100)
  bass_name = f"bass_shifted_{shifting_key}.wav"
  drums_name = f"drums_shifted_{shifting_key}.wav"
  other_name = f"other_shifted_{shifting_key}.wav"
else:
  bass_name = "bass.wav"
  drums_name = "drums.wav"
  other_name = "other.wav"

<ipython-input-12-7ec35281bdfc>:13: FutureWarning: Pass sr=44100 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  y_shifted = librosa.effects.pitch_shift(y, sample_rate, n_steps=shifting_key)


In [13]:
#@title 분리된 음원 합치기
from pydub import AudioSegment
#@markdown ## 보컬 음량 조절 (db)
db = 3 #@param {type:"integer"}
sound1 = AudioSegment.from_file(os.path.join(drive_infer_dir, bass_name), format="wav")
sound2 = AudioSegment.from_file(os.path.join(drive_infer_dir, drums_name), format="wav")
sound3 = AudioSegment.from_file(os.path.join(drive_infer_dir, other_name), format="wav")
sound4 = AudioSegment.from_file(os.path.join(drive_infer_dir,f"infered_output_key{key}.wav"), format="wav")
sound4 += db
# Overlay sound2 over sound1 at position 0  (use louder instead of sound1 to use the louder version)
overlay = sound1.overlay(sound2, position=0)
overlay = overlay.overlay(sound3, position=0)
overlay = overlay.overlay(sound4, position=0)

# simple export
file_handle = overlay.export(os.path.join(infer_dir,f"infered_final_key{key}.wav"), format="wav")
shutil.copy(os.path.join(infer_dir,f"infered_final_key{key}.wav"), drive_infer_dir)

'/content/drive/MyDrive/chatgpt안썻조/기계학습과딥러닝_이재욱교수님/Final_Project/diff-svc/inference/submission/madeleinelove_cut2_2023-05-05_05-57/infered_final_key-3.wav'

In [14]:
overlay

In [ ]:
overlay